<a id='1'></a>
# Import modules

In [1]:
%load_ext autoreload
%autoreload 2

import keras.backend as K

Using TensorFlow backend.


<a id='4'></a>
# Model Configuration

In [2]:
K.set_learning_phase(0)

W1009 10:38:14.825351 140615413020416 deprecation_wrapper.py:119] From /home/nguyen.thanh.trungb/anaconda3/envs/main_env_2/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:153: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [3]:
# Input/Output resolution
RESOLUTION = 64 # 64x64, 128x128, 256x256
assert (RESOLUTION % 64) == 0, "RESOLUTION should be 64, 128, 256"

In [4]:
# Architecture configuration
arch_config = {}
arch_config['IMAGE_SHAPE'] = (RESOLUTION, RESOLUTION, 3)
arch_config['use_self_attn'] = True
arch_config['norm'] = "instancenorm" # instancenorm, batchnorm, layernorm, groupnorm, none
arch_config['model_capacity'] = "standard" # standard, lite

<a id='5'></a>
# Define models

In [5]:
from networks.faceswap_gan_model import FaceswapGANModel

In [6]:
model = FaceswapGANModel(**arch_config)

W1009 10:38:26.506366 140615413020416 deprecation_wrapper.py:119] From /home/nguyen.thanh.trungb/anaconda3/envs/main_env_2/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1009 10:38:26.508164 140615413020416 deprecation_wrapper.py:119] From /home/nguyen.thanh.trungb/anaconda3/envs/main_env_2/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1009 10:38:26.514708 140615413020416 deprecation_wrapper.py:119] From /home/nguyen.thanh.trungb/anaconda3/envs/main_env_2/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1009 10:38:26.764809 140615413020416 deprecation_wrapper.py:119] From /home/nguyen.thanh.trungb/Desktop/FaceSwap/faceswap-GAN-2/networks/custom_inits/icnr_initiali

<a id='6'></a>
# Load Model Weights

In [7]:
model.load_weights(path="./models")

Model weights files are successfully loaded.


<a id='12'></a>
# Video Conversion

In [8]:
from detector.face_detector import MTCNNFaceDetector

mtcnn_weights_dir = "./mtcnn_weights/"
fd = MTCNNFaceDetector(sess=K.get_session(), model_path=mtcnn_weights_dir)


W1009 10:38:37.708941 140615413020416 deprecation.py:506] From /home/nguyen.thanh.trungb/anaconda3/envs/main_env_2/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1009 10:38:37.729477 140615413020416 deprecation_wrapper.py:119] From /home/nguyen.thanh.trungb/Desktop/FaceSwap/faceswap-GAN-2/mtcnn_detect_face.py:179: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1009 10:38:37.782938 140615413020416 deprecation.py:323] From /home/nguyen.thanh.trungb/Desktop/FaceSwap/faceswap-GAN-2/mtcnn_detect_face.py:215: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W1009 10

In [41]:
%load_ext autoreload
%autoreload 2

from converter.video_converter import VideoConverter


options = {
    # ===== Fixed =====
    "use_smoothed_bbox": True,
    "use_kalman_filter": True,
    "use_auto_downscaling": False,
    "bbox_moving_avg_coef": 0.65,
    "min_face_area": 35 * 35,
    "IMAGE_SHAPE": model.IMAGE_SHAPE,
    # ===== Tunable =====
    "kf_noise_coef": 3e-3,
    "use_color_correction": "none",
    "detec_threshold": 0.7,
    "roi_coverage": 0.9,
    "enhance": 0.,
    "output_type": 3,
    "direction": "AtoB",
}

vc = VideoConverter()
vc.set_face_detector(fd)
vc.set_gan_model(model)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Video conversion configuration


- `use_smoothed_bbox`: 
    - Boolean. Whether to enable smoothed bbox.
- `use_kalman_filter`: 
    - Boolean. Whether to enable Kalman filter.
- `use_auto_downscaling`:
    - Boolean. Whether to enable auto-downscaling in face detection (to prevent OOM error).
- `bbox_moving_avg_coef`: 
    - Float point between 0 and 1. Smoothing coef. used when use_kalman_filter is set False.
- `min_face_area`:
    - int x int. Minimum size of face. Detected faces smaller than min_face_area will not be transformed.
- `IMAGE_SHAPE`:
    - Input/Output resolution of the GAN model
- `kf_noise_coef`:
    - Float point. Increase by 10x if tracking is slow. Decrease by 1/10x if trakcing works fine but jitter occurs.
- `use_color_correction`: 
    - String of "adain", "adain_xyz", "hist_match", or "none". The color correction method to be applied.
- `detec_threshold`: 
    - Float point between 0 and 1. Decrease its value if faces are missed. Increase its value to reduce false positives.
- `roi_coverage`: 
    - Float point between 0 and 1 (exclusive). Center area of input images to be cropped (Suggested range: 0.85 ~ 0.95)
- `enhance`: 
    - Float point. A coef. for contrast enhancement in the region of alpha mask (Suggested range: 0. ~ 0.4)
- `output_type`: 
    - Layout format of output video: 1. [ result ], 2. [ source | result ], 3. [ source | result | mask ]
- `direction`: 
    - String of "AtoB" or "BtoA". Direction of face transformation.

In [42]:
input_fn = "/home/nguyen.thanh.trungb/Desktop/FaceSwap/faceswap-GAN-1/dataset/video/A.mp4"
output_fn = "OUTPUT_VIDEO.mp4"
duration = (10, 17) 

In [43]:
vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)

chunk:  22%|██▏       | 34/155 [00:00<00:00, 327.80it/s, now=None]

Moviepy - Building video OUTPUT_VIDEO.mp4.
MoviePy - Writing audio in OUTPUT_VIDEOTEMP_MPY_wvf_snd.mp3


t:   0%|          | 0/210 [00:00<?, ?it/s, now=None]               

MoviePy - Done.
Moviepy - Writing video OUTPUT_VIDEO.mp4



Moviepy - Done !
Moviepy - video ready OUTPUT_VIDEO.mp4
